In [35]:
%matplotlib inline
%config InlineBackend.figure_format = "retina"
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from torch import nn, optim
from torchvision import datasets, transforms, models, utils
import torch.nn.functional as F
from torch.autograd import Variable
import time
from glob import glob
from torch.utils.data.sampler import SubsetRandomSampler
from collections import OrderedDict
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
print(device)

cpu


In [36]:
data_path = "../data/raw/"

In [37]:
annotation_df = pd.read_csv(data_path+'annotation.csv')
annotation_df.head()
print("Annotation data: ",len(annotation_df))

Annotation data:  1026


In [38]:
all_files = glob(data_path+"images/*")
all_jpgs = sorted([img for img in all_files if ".jpg" in img or ".jpeg" in img or "JPG" in img])
print("Total {} photos ".format(len(all_jpgs)))

Total 1026 photos 


In [39]:
id_path = [image for image in all_jpgs ]
image_df = pd.DataFrame(id_path, columns=['path'])

In [40]:
full_df = image_dataframe.merge(profile_dataframe, left_index=True, right_index=True)
full_df.head()

,path,image,height,weight,BMI
0,../data/raw/images/f_001.jpg,f_001,1.55,61.0,25.390219
1,../data/raw/images/f_002.jpg,f_002,1.76,85.0,27.440599
2,../data/raw/images/f_003.jpg,f_003,1.78,56.0,17.674536
3,../data/raw/images/f_004.jpg,f_004,1.63,63.0,23.711845
4,../data/raw/images/f_005.jpg,f_005,1.76,54.0,17.432851


In [48]:
n = 65
image_path = full_df.iloc[n, 0]
image_name = full_df.iloc[n, 1]
values = np.asarray(full_df.iloc[n, 2:])
values = values.astype('float').reshape(-1, 3)

print('Image path: {}'.format(image_path))
print('Image name: {}'.format(image_name))
print('Values shape: {}'.format(values.shape))
print('height: {} m, weight: {} kg, bmi: {}'.format(*values[:, 0], *values[:, 1], *values[:, 2]))

Image path: ../data/raw/images/f_066.jpg
Image name: f_066
Values shape: (1, 3)
height: 1.77 m, weight: 55.0 kg, bmi: 17.55561939


In [ ]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image

class FaceToBMIDataset(Dataset):
    def __init__(self, csv_file, image_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.annotaion = pd.read_csv(csv_file)
        self.image_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotaion)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.image_dir, self.annotaion.iloc[idx, 0])
        image = io.imread(img_name)
        values = self.values_frame.iloc[idx, 1:]
        values = np.array([values])
        values = values.astype('float').reshape(-1, 3)
        item = {'image': image, 'values': values}

        if self.transform:
            sample = self.transform(sample)

        return sample